# Sprint 4 — Webinar 12 (Práctica)
**Modalidad:** Coding Together + Breakrooms  
**Tema:** Funnels y Cohorts en PostgreSQL  
**Versión:** 2025-11-02T15:27:26

> Esta sesión práctica parte de lo aprendido en **S4W11 (Teórico)** y lo lleva a ejercicios guiados. Usaremos un *nuevo* esquema para no mezclar con la clase anterior.


<div style="text-align: center">
    <img src="https://raw.githubusercontent.com/ljpiere/tpdata_python/main/images/w1s1_2.png" width="400">
</div>

## Estructura y timeboxing
- **0) Breakrooms (10 min):** Ejercicio simple para calentar usando `DISTINCT`, conteos y validaciones rápidas.
- **1) Coding Together (60 min):** Series de ejercicios guiados para construir funnels, calcular *drop-off*, y armar cohorts/retención.

> Recomendación: abrir conexión a PostgreSQL con `psql` o DBeaver antes de iniciar.


## 0) Preparación de entorno: nueva BD / esquema

- Tener un servidor PostgreSQL accesible (local o remoto).  
- Usuario con permisos de creación en un esquema de práctica.  
- Conexión por `psql`, un cliente gráfico (DBeaver), o extensiones como `ipython-sql` (opcional).
- Puedes usar la página web: https://sqliteonline.com/


In [ ]:
# --- DDL S4W12 (copia/ejecuta en PostgreSQL) ---
print(r'''
-- ==============================================
-- Esquema y tablas para la práctica (S4W12)
-- ==============================================

CREATE SCHEMA IF NOT EXISTS s4w12;

DROP TABLE IF EXISTS s4w12.events CASCADE;
DROP TABLE IF EXISTS s4w12.users  CASCADE;

CREATE TABLE s4w12.users (
  user_id   BIGSERIAL PRIMARY KEY,
  signup_ts TIMESTAMP NOT NULL,
  plan      TEXT CHECK (plan IN ('free','paid')) NOT NULL,
  channel   TEXT CHECK (channel IN ('organic','ads','referral','email')) NOT NULL,
  device    TEXT CHECK (device IN ('web','android','ios')) NOT NULL
);

CREATE TABLE s4w12.events (
  event_id   BIGSERIAL PRIMARY KEY,
  user_id    BIGINT REFERENCES s4w12.users(user_id),
  event_name TEXT NOT NULL,
  event_ts   TIMESTAMP NOT NULL,
  props      JSONB DEFAULT '{}'::jsonb
);

CREATE INDEX IF NOT EXISTS idx_s4w12_events_user_ts ON s4w12.events (user_id, event_ts);
CREATE INDEX IF NOT EXISTS idx_s4w12_events_name_ts ON s4w12.events (event_name, event_ts);
CREATE INDEX IF NOT EXISTS idx_s4w12_events_props   ON s4w12.events USING GIN (props);
''')

In [ ]:
# --- Seed S4W12 (copia/ejecuta en PostgreSQL) ---
print(r'''
-- ==========================================================
-- Carga mínima con más variabilidad (enero-marzo 2021)
-- ==========================================================
INSERT INTO s4w12.users (signup_ts, plan, channel, device) VALUES
  -- Enero cohort
  ('2021-01-02 09:10','free','organic','web'),
  ('2021-01-03 14:20','paid','ads','ios'),
  ('2021-01-05 08:05','free','referral','android'),
  ('2021-01-10 18:42','free','organic','web'),
  ('2021-01-12 11:11','paid','email','web'),
  -- Febrero cohort
  ('2021-02-01 09:15','free','ads','android'),
  ('2021-02-05 16:00','paid','referral','ios'),
  ('2021-02-09 10:30','free','organic','web'),
  ('2021-02-12 21:45','free','email','android'),
  ('2021-02-15 07:50','paid','ads','web'),
  -- Marzo cohort
  ('2021-03-02 10:00','free','ads','android'),
  ('2021-03-04 12:00','paid','email','ios'),
  ('2021-03-05 15:20','free','organic','web'),
  ('2021-03-08 18:10','paid','referral','web'),
  ('2021-03-10 09:40','free','organic','android');

-- Flujo típico con ruidos: session_start -> view_item -> add_to_cart -> begin_checkout -> purchase
-- Enero: ids 1..5
INSERT INTO s4w12.events (user_id, event_name, event_ts, props) VALUES
  (1,'session_start','2021-01-02 09:12','{"utm":"organic"}'),
  (1,'view_item','2021-01-02 09:14','{"sku":"A101"}'),
  (1,'add_to_cart','2021-01-02 09:15','{"sku":"A101"}'),
  (1,'begin_checkout','2021-01-02 09:16','{"cart_value":29.9}'),
  (1,'purchase','2021-01-02 09:18','{"order_id":"O-0001","value":29.9}'),

  (2,'session_start','2021-01-03 14:21','{"utm":"ads"}'),
  (2,'view_item','2021-01-03 14:22','{"sku":"B222"}'),
  (2,'add_to_cart','2021-01-03 14:23','{"sku":"B222"}'),
  (2,'begin_checkout','2021-01-03 14:24','{"cart_value":58.0}'),
  (2,'purchase','2021-01-03 14:27','{"order_id":"O-0002","value":58.0}'),

  (3,'session_start','2021-01-05 08:06','{"utm":"referral"}'),
  (3,'view_item','2021-01-05 08:07','{"sku":"C333"}'),
  (3,'add_to_cart','2021-01-05 08:08','{"sku":"C333"}'),
  (3,'begin_checkout','2021-01-05 08:09','{"cart_value":105.0}'),
  -- abandona antes de purchase

  (4,'session_start','2021-01-10 18:43','{"utm":"organic"}'),
  (4,'view_item','2021-01-10 18:46','{"sku":"D444"}'),
  -- no add_to_cart

  (5,'session_start','2021-01-12 11:12','{"utm":"email"}'),
  (5,'view_item','2021-01-12 11:13','{"sku":"A101"}'),
  (5,'add_to_cart','2021-01-12 11:14','{"sku":"A101"}');
  -- no checkout/purchase

-- Febrero: ids 6..10
INSERT INTO s4w12.events (user_id, event_name, event_ts, props) VALUES
  (6,'session_start','2021-02-01 09:17','{"utm":"ads"}'),
  (6,'view_item','2021-02-01 09:18','{"sku":"E555"}'),
  -- no add_to_cart

  (7,'session_start','2021-02-05 16:01','{"utm":"referral"}'),
  (7,'view_item','2021-02-05 16:02','{"sku":"B222"}'),
  (7,'add_to_cart','2021-02-05 16:03','{"sku":"B222"}'),
  (7,'begin_checkout','2021-02-05 16:04','{"cart_value":58.0}'),
  (7,'purchase','2021-02-05 16:05','{"order_id":"O-0007","value":58.0}'),

  (8,'session_start','2021-02-09 10:31','{"utm":"organic"}'),
  (8,'view_item','2021-02-09 10:32','{"sku":"C333"}'),
  (8,'add_to_cart','2021-02-09 10:34','{"sku":"C333"}'),
  -- no checkout

  (9,'session_start','2021-02-12 21:46','{"utm":"email"}'),
  -- rebote

  (10,'session_start','2021-02-15 07:51','{"utm":"ads"}'),
  (10,'view_item','2021-02-15 07:52','{"sku":"A101"}'),
  (10,'add_to_cart','2021-02-15 07:53','{"sku":"A101"}'),
  (10,'begin_checkout','2021-02-15 07:54','{"cart_value":29.9}'),
  (10,'purchase','2021-02-15 07:56','{"order_id":"O-0010","value":29.9}');

-- Marzo: ids 11..15 (agregamos algo de ruido/datos incompletos)
INSERT INTO s4w12.events (user_id, event_name, event_ts, props) VALUES
  (11,'session_start','2021-03-02 10:02','{"utm":"ads"}'),
  (11,'view_item','2021-03-02 10:05','{"sku":"E555"}'),
  (11,'add_to_cart','2021-03-02 10:06','{"sku":"E555"}'),
  (11,'begin_checkout','2021-03-02 10:07','{"cart_value":58.0}'),
  (11,'purchase','2021-03-02 10:10','{"order_id":"O-0011","value":58.0}'),

  (12,'session_start','2021-03-04 12:02','{"utm":"email"}'),
  (12,'view_item','2021-03-04 12:04','{"sku":"A101"}'),
  -- no add_to_cart

  (13,'session_start','2021-03-05 15:22','{"utm":"organic"}'),
  (13,'view_item','2021-03-05 15:24','{"sku":"C333"}'),
  (13,'add_to_cart','2021-03-05 15:26','{"sku":"C333"}'),
  (13,'begin_checkout','2021-03-05 15:28','{"cart_value":105.0}'),
  -- abandona

  (14,'session_start','2021-03-08 18:12','{"utm":"referral"}'),
  -- rebote

  (15,'session_start','2021-03-10 09:42','{"utm":"organic"}'),
  (15,'view_item','2021-03-10 09:44','{"sku":"A101"}');
''')

# Ejercicio 0 — Breakrooms (10 minutos)
**Objetivo:** calentar motores con tareas simples usando lo visto en S4W11.

**Tareas:**
1. Lista los `event_name` distintos en `s4w12.events` (orden alfabético).
2. Cuenta cuántos **usuarios únicos** tuvieron `session_start` en **marzo 2021**.
3. Identifica usuarios que **agregaron al carrito** pero **no** llegaron a `begin_checkout`.

> Trabaja en parejas/triadas. Comparen respuestas y anoten dudas de SQL/interpretación.

**Pistas:**
- **0.1 DISTINCT:** recuerda que `SELECT DISTINCT col FROM tabla` elimina duplicados. Ordena con `ORDER BY 1` para asegurar legibilidad.  
- **0.2 Conteo de usuarios:** usa `COUNT(DISTINCT user_id)` y limita por rango de fechas. **Pista:** `event_ts >= 'YYYY-MM-01' AND event_ts < 'YYYY-MM-01' + 1 month` evita problemas de fin de mes.  
- **0.3 Set difference:** para “tuvo A pero no B”, crea dos CTEs con `DISTINCT user_id` y haz `LEFT JOIN ... WHERE k.user_id IS NULL`.

**Errores comunes:**  
1) Olvidar el filtro temporal (resultados cambian).  
2) Contar filas en vez de usuarios (`COUNT(*)` vs `COUNT(DISTINCT user_id)`).  
3) Usar `BETWEEN` con fechas puede incluir bordes no deseados; prefiere `>=` y `<`.

In [ ]:
# --- Solución sugerida: Ejercicio 0 ---
print(r'''
-- 0.1 tipos de evento únicos
SELECT DISTINCT event_name
FROM s4w12.events
ORDER BY 1;

-- 0.2 usuarios únicos con session_start en marzo 2021
SELECT COUNT(DISTINCT user_id) AS users_mar_session
FROM s4w12.events
WHERE event_name = 'session_start'
  AND event_ts >= '2021-03-01'::timestamp
  AND event_ts <  '2021-04-01'::timestamp;

-- 0.3 usuarios con add_to_cart pero sin begin_checkout
WITH cart AS (
  SELECT DISTINCT user_id FROM s4w12.events WHERE event_name='add_to_cart'
), chk AS (
  SELECT DISTINCT user_id FROM s4w12.events WHERE event_name='begin_checkout'
)
SELECT c.user_id
FROM cart c
LEFT JOIN chk  k ON k.user_id = c.user_id
WHERE k.user_id IS NULL
ORDER BY 1;
''')

# 1) Coding Together (~60 min)

Iremos construyendo consultas paso a paso. Para cada ejercicio hay un **starter** y una **solución**.


## 1.1 Funnel por etapas con CTEs (Feb 2021)
**Meta:** `session_start → view_item → add_to_cart → begin_checkout → purchase`

- Divide en CTEs por etapa; cada CTE devuelve **usuarios distintos** que alcanzaron esa etapa.  
- Asegúrate de filtrar por el **mismo rango temporal** en el `base`.  
- Para comparar etapas, recuerda que un usuario que llegó a `purchase` ya pasó por las anteriores, pero **tu conteo por CTE** no fuerza esa relación; por eso usamos `DISTINCT` y comparamos tamaños.  
- **Pista de validación:** agrega `EXCEPT` entre CTEs para listar usuarios que **saltaron** una etapa (datos inconsistentes) y discutir si es posible por negocio o por calidad de datos.

**Starter (rellena o ejecuta y adapta):**


In [ ]:
# --- Starter 1.1 ---
print(r'''
-- Starter 1.1: CTEs por etapa (Feb 2021)
WITH base AS (
  SELECT * FROM s4w12.events
  WHERE event_ts >= '2021-02-01'::timestamp
    AND event_ts <  '2021-03-01'::timestamp
), s AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='session_start'
), v AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='view_item'
), c AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='add_to_cart'
), ch AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='begin_checkout'
), p AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='purchase'
)
SELECT
  (SELECT COUNT(*) FROM s)  AS n_session,
  (SELECT COUNT(*) FROM v)  AS n_view,
  (SELECT COUNT(*) FROM c)  AS n_cart,
  (SELECT COUNT(*) FROM ch) AS n_checkout,
  (SELECT COUNT(*) FROM p)  AS n_purchase;
''')

In [ ]:
# --- Solución 1.1 ---
print(r'''
-- Solución 1.1 (idéntica al starter; discutir resultados y dudas)
WITH base AS (
  SELECT * FROM s4w12.events
  WHERE event_ts >= '2021-02-01'::timestamp
    AND event_ts <  '2021-03-01'::timestamp
), s AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='session_start'
), v AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='view_item'
), c AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='add_to_cart'
), ch AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='begin_checkout'
), p AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='purchase'
), counts AS (
  SELECT
    (SELECT COUNT(*) FROM s)  AS n_session,
    (SELECT COUNT(*) FROM v)  AS n_view,
    (SELECT COUNT(*) FROM c)  AS n_cart,
    (SELECT COUNT(*) FROM ch) AS n_checkout,
    (SELECT COUNT(*) FROM p)  AS n_purchase
)
SELECT
  n_session,n_view,n_cart,n_checkout,n_purchase,
  (n_session - n_view)       AS drop_s_v,
  (n_view - n_cart)          AS drop_v_c,
  (n_cart - n_checkout)      AS drop_c_ch,
  (n_checkout - n_purchase)  AS drop_ch_p,
  ROUND(100.0 * n_purchase / NULLIF(n_session,0),2) AS conv_s_to_p_pct
FROM counts;
''')

## 1.2 Tiempos entre etapas + cuello de botella
**Meta:** calcular diferencias de tiempo medianas entre pasos y detectar la etapa con mayor *drop-off*.

**Starter (usa window functions y agregaciones):**

- Calcula el **primer timestamp** de cada evento por usuario (`MIN(event_ts)` por `event_name`).  
- Usa diferencias de tiempo en **minutos** para facilitar interpretación de *latencias operativas*.  
- La **mediana (P50)** es más robusta que el promedio ante outliers.  
- **Pista:** si te sale `NULL` en una resta, el usuario no pasó por ambas etapas; puedes excluir con `WHERE ts_a IS NOT NULL AND ts_b IS NOT NULL` o dejar `NULL` y depender de la mediana que ignora nulos.

In [ ]:
# --- Starter 1.2 ---
print(r'''
-- Starter 1.2: tiempo entre etapas por usuario (ejemplo con s->v y v->c en feb-2021)
WITH base AS (
  SELECT * FROM s4w12.events
  WHERE event_ts >= '2021-02-01'::timestamp
    AND event_ts <  '2021-03-01'::timestamp
),
s AS (SELECT user_id, MIN(event_ts) AS ts FROM base WHERE event_name='session_start' GROUP BY 1),
v AS (SELECT user_id, MIN(event_ts) AS ts FROM base WHERE event_name='view_item'     GROUP BY 1),
c AS (SELECT user_id, MIN(event_ts) AS ts FROM base WHERE event_name='add_to_cart'   GROUP BY 1),
ch AS(SELECT user_id, MIN(event_ts) AS ts FROM base WHERE event_name='begin_checkout'GROUP BY 1),
p AS (SELECT user_id, MIN(event_ts) AS ts FROM base WHERE event_name='purchase'      GROUP BY 1)
SELECT
  COALESCE(s.user_id,v.user_id,c.user_id,ch.user_id,p.user_id) AS user_id,
  EXTRACT(EPOCH FROM (v.ts  - s.ts))/60.0 AS min_s_to_v,
  EXTRACT(EPOCH FROM (c.ts  - v.ts))/60.0 AS min_v_to_c,
  EXTRACT(EPOCH FROM (ch.ts - c.ts))/60.0 AS min_c_to_ch,
  EXTRACT(EPOCH FROM (p.ts  - ch.ts))/60.0 AS min_ch_to_p
FROM s
FULL JOIN v  USING (user_id)
FULL JOIN c  USING (user_id)
FULL JOIN ch USING (user_id)
FULL JOIN p  USING (user_id);
''')

In [ ]:
# --- Solución 1.2 ---
print(r'''
-- Solución 1.2: medianas + cuello de botella por drop-off
WITH base AS (
  SELECT * FROM s4w12.events
  WHERE event_ts >= '2021-02-01'::timestamp
    AND event_ts <  '2021-03-01'::timestamp
),
s AS (SELECT DISTINCT user_id FROM base WHERE event_name='session_start'),
v AS (SELECT DISTINCT user_id FROM base WHERE event_name='view_item'),
c AS (SELECT DISTINCT user_id FROM base WHERE event_name='add_to_cart'),
ch AS(SELECT DISTINCT user_id FROM base WHERE event_name='begin_checkout'),
p AS (SELECT DISTINCT user_id FROM base WHERE event_name='purchase'),
counts AS (
  SELECT
    (SELECT COUNT(*) FROM s)  AS n_s,
    (SELECT COUNT(*) FROM v)  AS n_v,
    (SELECT COUNT(*) FROM c)  AS n_c,
    (SELECT COUNT(*) FROM ch) AS n_ch,
    (SELECT COUNT(*) FROM p)  AS n_p
),
times AS (
  SELECT
    u.user_id,
    MIN(CASE WHEN event_name='session_start'  THEN event_ts END) AS ts_s,
    MIN(CASE WHEN event_name='view_item'      THEN event_ts END) AS ts_v,
    MIN(CASE WHEN event_name='add_to_cart'    THEN event_ts END) AS ts_c,
    MIN(CASE WHEN event_name='begin_checkout' THEN event_ts END) AS ts_ch,
    MIN(CASE WHEN event_name='purchase'       THEN event_ts END) AS ts_p
  FROM base u
  GROUP BY u.user_id
), diffs AS (
  SELECT
    user_id,
    EXTRACT(EPOCH FROM (ts_v  - ts_s))/60.0 AS min_s_v,
    EXTRACT(EPOCH FROM (ts_c  - ts_v))/60.0 AS min_v_c,
    EXTRACT(EPOCH FROM (ts_ch - ts_c))/60.0 AS min_c_ch,
    EXTRACT(EPOCH FROM (ts_p  - ts_ch))/60.0 AS min_ch_p
  FROM times
)
SELECT
  c.n_s, c.n_v, c.n_c, c.n_ch, c.n_p,
  (n_s - n_v)  AS drop_s_v,
  (n_v - n_c)  AS drop_v_c,
  (n_c - n_ch) AS drop_c_ch,
  (n_ch - n_p) AS drop_ch_p,
  ROUND(100.0 * n_p / NULLIF(n_s,0),2) AS conv_total_pct,
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY min_s_v)  AS p50_min_s_v,
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY min_v_c)  AS p50_min_v_c,
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY min_c_ch) AS p50_min_c_ch,
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY min_ch_p) AS p50_min_ch_p
FROM counts c, diffs;
''')

## 1.3 Cohorts semanales y retención (0..6 semanas)
**Meta:** asignar cohort por `date_trunc('week', signup_ts)` y medir retención semanal (0..6).

- **Definición de cohort:** primera fecha de contacto (`signup_ts`) o primer evento (`MIN(event_ts)`) según el caso. Aquí usamos `signup_ts`.  
- Construye una **malla temporal (grid)** con `generate_series(0,6)` para *week_offset* y une con cada usuario.  
- **Regla de pertenencia:** un usuario está “retenido” en `week_offset = k` si tiene **algún evento** dentro del intervalo `[cohort_week + k, cohort_week + k + 1)`.  
- **Pista:** valida que el denominador (tamaño de cohort) sea constante por fila.


**Starter:**

In [ ]:
# --- Starter 1.3 ---
print(r'''
-- Starter 1.3: cohort semanal + tabla ancha para heatmap
WITH u AS (
  SELECT user_id, date_trunc('week', signup_ts)::date AS cohort_week
  FROM s4w12.users
),
grid AS (SELECT generate_series(0,6) AS w),
ret AS (
  SELECT
    u.cohort_week,
    g.w AS week_offset,
    COUNT(DISTINCT u.user_id) FILTER (
      WHERE EXISTS (
        SELECT 1 FROM s4w12.events e
        WHERE e.user_id = u.user_id
          AND e.event_ts::date >= (u.cohort_week + (g.w || ' week')::interval)::date
          AND e.event_ts::date <  (u.cohort_week + ((g.w+1) || ' week')::interval)::date
      )
    )::float AS retained,
    COUNT(DISTINCT u.user_id)::float AS cohort_users
  FROM u CROSS JOIN grid g
  GROUP BY u.cohort_week, g.w
)
SELECT
  cohort_week,
  MAX(CASE WHEN week_offset=0 THEN ROUND(100.0*retained/cohort_users,2) END) AS w0,
  MAX(CASE WHEN week_offset=1 THEN ROUND(100.0*retained/cohort_users,2) END) AS w1,
  MAX(CASE WHEN week_offset=2 THEN ROUND(100.0*retained/cohort_users,2) END) AS w2,
  MAX(CASE WHEN week_offset=3 THEN ROUND(100.0*retained/cohort_users,2) END) AS w3,
  MAX(CASE WHEN week_offset=4 THEN ROUND(100.0*retained/cohort_users,2) END) AS w4,
  MAX(CASE WHEN week_offset=5 THEN ROUND(100.0*retained/cohort_users,2) END) AS w5,
  MAX(CASE WHEN week_offset=6 THEN ROUND(100.0*retained/cohort_users,2) END) AS w6
FROM ret
GROUP BY cohort_week
ORDER BY cohort_week;
''')

In [ ]:
# --- Solución 1.3 (igual al starter; discutir lectura de la tabla) ---
print(r'''
-- Starter 1.3: cohort semanal + tabla ancha para heatmap
WITH u AS (
  SELECT user_id, date_trunc('week', signup_ts)::date AS cohort_week
  FROM s4w12.users
),
grid AS (SELECT generate_series(0,6) AS w),
ret AS (
  SELECT
    u.cohort_week,
    g.w AS week_offset,
    COUNT(DISTINCT u.user_id) FILTER (
      WHERE EXISTS (
        SELECT 1 FROM s4w12.events e
        WHERE e.user_id = u.user_id
          AND e.event_ts::date >= (u.cohort_week + (g.w || ' week')::interval)::date
          AND e.event_ts::date <  (u.cohort_week + ((g.w+1) || ' week')::interval)::date
      )
    )::float AS retained,
    COUNT(DISTINCT u.user_id)::float AS cohort_users
  FROM u CROSS JOIN grid g
  GROUP BY u.cohort_week, g.w
)
SELECT
  cohort_week,
  MAX(CASE WHEN week_offset=0 THEN ROUND(100.0*retained/cohort_users,2) END) AS w0,
  MAX(CASE WHEN week_offset=1 THEN ROUND(100.0*retained/cohort_users,2) END) AS w1,
  MAX(CASE WHEN week_offset=2 THEN ROUND(100.0*retained/cohort_users,2) END) AS w2,
  MAX(CASE WHEN week_offset=3 THEN ROUND(100.0*retained/cohort_users,2) END) AS w3,
  MAX(CASE WHEN week_offset=4 THEN ROUND(100.0*retained/cohort_users,2) END) AS w4,
  MAX(CASE WHEN week_offset=5 THEN ROUND(100.0*retained/cohort_users,2) END) AS w5,
  MAX(CASE WHEN week_offset=6 THEN ROUND(100.0*retained/cohort_users,2) END) AS w6
FROM ret
GROUP BY cohort_week
ORDER BY cohort_week;
''')

## 1.4 Retención segmentada (free vs paid)
**Meta:** comparar retención por `plan` y comentar hallazgos.

- Agrega atributos de usuario (`plan`, `channel`, `device`) a la tabla de cohort para desglosar la retención.  
- Compara **tendencias**, no solo números absolutos: ¿`paid` retiene mejor en W1 pero converge en W3?  
- **Pista:** usa `ORDER BY cohort_week, plan, week_offset` y exporta a CSV para armar dos heatmaps lado a lado en Sheets.

**Starter:**


In [ ]:
# --- Starter 1.4 ---
print(r'''
-- Starter 1.4: retención semanal segmentada por plan
WITH u AS (
  SELECT user_id,
         date_trunc('week', signup_ts)::date AS cohort_week,
         plan
  FROM s4w12.users
),
grid AS (SELECT generate_series(0,6) AS w),
ret AS (
  SELECT
    u.cohort_week, u.plan, g.w AS week_offset,
    COUNT(DISTINCT u.user_id) FILTER (
      WHERE EXISTS (
        SELECT 1 FROM s4w12.events e
        WHERE e.user_id = u.user_id
          AND e.event_ts::date >= (u.cohort_week + (g.w || ' week')::interval)::date
          AND e.event_ts::date <  (u.cohort_week + ((g.w+1) || ' week')::interval)::date
      )
    )::float AS retained,
    COUNT(DISTINCT u.user_id)::float AS cohort_users
  FROM u CROSS JOIN grid g
  GROUP BY u.cohort_week, u.plan, g.w
)
SELECT cohort_week, plan, week_offset,
       CASE WHEN cohort_users=0 THEN 0
            ELSE ROUND(100.0*retained/cohort_users,2) END AS retention_pct
FROM ret
ORDER BY cohort_week, plan, week_offset;
''')

In [ ]:
# --- Solución 1.4 (igual al starter) ---
print(r'''
-- Starter 1.4: retención semanal segmentada por plan
WITH u AS (
  SELECT user_id,
         date_trunc('week', signup_ts)::date AS cohort_week,
         plan
  FROM s4w12.users
),
grid AS (SELECT generate_series(0,6) AS w),
ret AS (
  SELECT
    u.cohort_week, u.plan, g.w AS week_offset,
    COUNT(DISTINCT u.user_id) FILTER (
      WHERE EXISTS (
        SELECT 1 FROM s4w12.events e
        WHERE e.user_id = u.user_id
          AND e.event_ts::date >= (u.cohort_week + (g.w || ' week')::interval)::date
          AND e.event_ts::date <  (u.cohort_week + ((g.w+1) || ' week')::interval)::date
      )
    )::float AS retained,
    COUNT(DISTINCT u.user_id)::float AS cohort_users
  FROM u CROSS JOIN grid g
  GROUP BY u.cohort_week, u.plan, g.w
)
SELECT cohort_week, plan, week_offset,
       CASE WHEN cohort_users=0 THEN 0
            ELSE ROUND(100.0*retained/cohort_users,2) END AS retention_pct
FROM ret
ORDER BY cohort_week, plan, week_offset;
''')

## 1.5 Bonus: Vista para funnel mensual
**Meta:** dejar una vista reutilizable que calcule conteos por etapa y conversión mensual.

- Mantén las reglas del negocio en una **vista**: rango temporal, normalización mensual, definición de cada etapa.  
- Puedes crear una **materialized view** si el dataset crece y luego `REFRESH MATERIALIZED VIEW` en ventanas.

In [ ]:
# --- Solución 1.5 ---
print(r'''
-- Bonus: crear una vista (o materialized view) de funnel mensual
DROP VIEW IF EXISTS s4w12.v_funnel_monthly;
CREATE VIEW s4w12.v_funnel_monthly AS
WITH base AS (
  SELECT *, date_trunc('month', event_ts)::date AS m
  FROM s4w12.events
),
s AS (SELECT m, COUNT(DISTINCT user_id) AS n FROM base WHERE event_name='session_start'  GROUP BY 1),
v AS (SELECT m, COUNT(DISTINCT user_id) AS n FROM base WHERE event_name='view_item'      GROUP BY 1),
c AS (SELECT m, COUNT(DISTINCT user_id) AS n FROM base WHERE event_name='add_to_cart'    GROUP BY 1),
ch AS(SELECT m, COUNT(DISTINCT user_id) AS n FROM base WHERE event_name='begin_checkout' GROUP BY 1),
p AS (SELECT m, COUNT(DISTINCT user_id) AS n FROM base WHERE event_name='purchase'       GROUP BY 1)
SELECT
  s.m AS month,
  s.n AS n_session,
  v.n AS n_view,
  c.n AS n_cart,
  ch.n AS n_checkout,
  p.n AS n_purchase,
  ROUND(100.0 * p.n / NULLIF(s.n,0),2) AS conv_s_to_p_pct
FROM s
LEFT JOIN v  USING (m)
LEFT JOIN c  USING (m)
LEFT JOIN ch USING (m)
LEFT JOIN p  USING (m)
ORDER BY month;
''')

## 1.6 Checks rápidos de calidad de datos
**Meta:** detectar duplicados exactos, eventos en el futuro y usuarios con etapas intermedias perdidas.

- **Duplicados exactos:** misma combinación `(user_id, event_name, event_ts)` más de una vez.  
- **Tiempos imposibles:** `event_ts > NOW()` o retrocede respecto de etapas previas.  
- **Saltos de etapa:** `purchase` sin `begin_checkout` (según negocio).  
- **Pista:** guarda evidencias con `CREATE TABLE s4w12.qa_findings AS (...query...)` para revisar luego.

In [ ]:
# --- Solución 1.6 ---
print(r'''
-- Duplicados exactos por usuario/ts/evento
SELECT user_id, event_name, event_ts, COUNT(*) AS n
FROM s4w12.events
GROUP BY 1,2,3
HAVING COUNT(*) > 1
ORDER BY n DESC, event_ts;

-- Eventos en el futuro
SELECT * FROM s4w12.events WHERE event_ts > NOW();

-- Usuarios con add_to_cart pero sin begin_checkout
WITH cart AS (
  SELECT DISTINCT user_id FROM s4w12.events WHERE event_name='add_to_cart'
), chk AS (
  SELECT DISTINCT user_id FROM s4w12.events WHERE event_name='begin_checkout'
)
SELECT c.user_id
FROM cart c
LEFT JOIN chk k USING (user_id)
WHERE k.user_id IS NULL
ORDER BY 1;
''')

## Exportar a CSV y visualizar en Google Sheets
Para *heatmap* de retención semanal (ej. 1.3):
```sql
COPY (
  /* Pega aquí la consulta de la tabla ancha (starter 1.3) */
) TO '/tmp/s4w12_retention_weekly.csv' WITH CSV HEADER;
```
Sube a Sheets → **Formato condicional → Escala de color**.


## Cierre
**Kahoot de repaso (5 min)**
- Guarda tus consultas clave en un script comentado.
- Exporta una tabla final y redacta una recomendación ejecutiva.
- **Tarea:** rehacer los ejercicios usando tus propias tablas reales si existen.

## Siguientes Pasos
- **Próxima sesión:** Iniciaremos con la siguiente seccion del sprint 4.
- **Participación continua:** asistir a Co-Learning y a Sprint Focus, y usar los canales de Discord para hacer preguntas.
- **Recordatorios:** la grabación y recursos utilizados, se comparten al finalizar la sesión; en caso de necesitar apoyo adicional, agenda un 1:1.


---
**Licencia:** CC BY 4.0 — Uso educativo.  
**Créditos:** Preparado para Sprint 4 — Webinar 12 (Práctica).
